# Clustering Crypto

In [131]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [132]:
# Load the crypto_data.csv dataset.
file_path = "crypto_data.csv"
crypto_df=pd.read_csv(file_path, index_col=0)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [133]:
# Keep all the cryptocurrencies that are being traded.
crypto_trade_df= pd.DataFrame(crypto_df[(crypto_df['IsTrading']==True)])
crypto_trade_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [134]:
# Remove the "IsTrading" column. 
crypto_trade_df.drop('IsTrading', axis=1, inplace=True)
crypto_trade_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
SERO,Super Zero,Ethash,PoW,NaN,1000000000
UOS,UOS,SHA-256,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [135]:
# Remove rows that have at least 1 null value.
crypto_trade_df = crypto_trade_df.dropna()

In [136]:
# Keep the rows where coins are mined.
crypto_trade_df = crypto_trade_df[crypto_trade_df['TotalCoinsMined']!=0]
crypto_trade_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [137]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_name_df= crypto_trade_df.filter(['CoinName'], axis=1)
crypto_name_df

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex
ZEN,Horizen


In [138]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_trade_df=crypto_trade_df.drop(columns=['CoinName'])
crypto_trade_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Equihash,PoW,7.296538e+06,21000000


In [139]:
# Use get_dummies() to create variables for text features.
X=pd.get_dummies(data=crypto_trade_df, columns=['Algorithm','ProofType'])
X

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEN,7.296538e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [140]:
# Standardize the data with StandardScaler().
scaled_crypto=StandardScaler().fit_transform(X)

### Deliverable 2: Reducing Data Dimensions Using PCA

In [141]:
# Using PCA to reduce dimension to three principal components.
pca=PCA(n_components=3)

In [142]:
# Create a DataFrame with the three principal components.
pca_crypto=pca.fit_transform(scaled_crypto)
scaled_crypto_df= pd.DataFrame(
    data=pca_crypto, columns=['PC 1', 'PC 2', 'PC 3'],
    index= crypto_trade_df.index)
scaled_crypto_df.head()

,PC 1,PC 2,PC 3
42,-0.333065,1.013553,-0.514650
404,-0.316449,1.013670,-0.514984
1337,2.279458,1.608250,-0.626878
BTC,-0.153590,-1.307634,0.153600
ETH,-0.145077,-2.005435,0.310537


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [143]:
# Create an elbow curve to find the best value for K.
inert=[]
k=list(range(1,11))

for i in k:
        kmean=KMeans(n_clusters=i, random_state=0)
        kmean.fit(scaled_crypto_df)
        inert.append(kmean.inertia_)

elbow_data={'k':k,'inertia':inert}
df_elbow=pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x='k',y='inertia',xticks=k,title="Elbow Curve")

C:\Users\patri\anaconda3\envs\ML\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [144]:
# Initialize the K-Means model.
KMmodel=KMeans(n_clusters=4,random_state=0)

# Fit the model
KMmodel.fit(scaled_crypto_df)

# Predict clusters
Predict = KMmodel.predict(scaled_crypto_df)

In [145]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
dfs=[crypto_trade_df, scaled_crypto_df]
clustered_df=pd.concat(dfs, join='outer', axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df['CoinName']=crypto_name_df['CoinName']

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df['Class'] = KMmodel.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(533, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.333065,1.013553,-0.514650,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.316449,1.013670,-0.514984,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.279458,1.608250,-0.626878,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.153590,-1.307634,0.153600,Bitcoin,3
ETH,Ethash,PoW,1.076842e+08,0,-0.145077,-2.005435,0.310537,Ethereum,3
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.152794,-1.124730,-0.008679,Litecoin,3
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.414673,1.212440,-0.435680,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.139154,-2.260396,0.212318,Monero,3
ETC,Ethash,PoW,1.133597e+08,210000000,-0.143524,-2.005537,0.310521,Ethereum Classic,3
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.127778,-2.031709,0.406029,ZCash,3


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [146]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig= px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    hover_name='CoinName',
    hover_data=['Algorithm'],
    width=800
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [147]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm','ProofType','TotalCoinSupply','TotalCoinsMined','Class'])

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [148]:
# Print the total number of tradable cryptocurrencies.
CoinSupplyTotal=clustered_df['TotalCoinSupply'].sum()
CryptoSupplyTotal=len(clustered_df)
print(f'Total tradable cryptocurrencies:{CryptoSupplyTotal}')

Total tradable cryptocurrencies:533


In [149]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
#drop columns with strings
#dfX=clustered_df.drop(columns=['Algorithm','ProofType','CoinName'])

#make dummies and get x-scale

scaledX=MinMaxScaler().fit_transform(clustered_df.loc[:,['TotalCoinSupply','TotalCoinsMined']])
scaledX

array([[4.20000000e-11, 5.94230127e-03],
       [5.32000000e-04, 7.00182308e-03],
       [3.14159265e-01, 3.53420682e-02],
       ...,
       [1.40022261e-03, 6.92655266e-03],
       [2.10000000e-05, 5.94962775e-03],
       [1.00000000e-06, 5.94243008e-03]])

In [150]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
Kmodel=KMeans(n_clusters=3,random_state=78).fit(scaledX)
column_names=['TotalCoinSupply','TotalCoinsMined']
scaled_clustered_df=pd.DataFrame(data=scaledX, index=clustered_df.index, columns=column_names)


# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
scaled_clustered_df['CoinName']=clustered_df['CoinName']

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
scaled_clustered_df['Class']=clustered_df['Class']

scaled_clustered_df.head(10)

,TotalCoinsMined,TotalCoinSupply,CoinName,Class
42,4.200000e-11,0.005942,42 Coin,0
404,5.320000e-04,0.007002,404Coin,0
1337,3.141593e-01,0.035342,EliteCoin,0
BTC,2.100000e-05,0.005960,Bitcoin,3
ETH,0.000000e+00,0.006050,Ethereum,3
LTC,8.400000e-05,0.006006,Litecoin,3
DASH,2.200000e-05,0.005951,Dash,0
XMR,0.000000e+00,0.005960,Monero,3
ETC,2.100000e-04,0.006056,Ethereum Classic,3
ZEC,2.100000e-05,0.005950,ZCash,3


In [154]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
scaled_clustered_df.hvplot.scatter(y='TotalCoinsMined', x='TotalCoinSupply', by="Class", hover_cols=['CoinName'])

:NdOverlay   [Class]
   :Scatter   [TotalCoinSupply]   (TotalCoinsMined,CoinName)